In [2]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')

from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
import pyspark
import pandas as pd
from pyspark.sql import *
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('Data_Understanding').getOrCreate()
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.types import StructType, StructField, IntegerType
import numpy as np
from pyspark.sql import functions as F




pandasDF = pd.read_csv('/home/ubuntu/BDAS_yang124/Datasets/College.csv')
sparkDF = spark.createDataFrame(pandasDF)
df = sparkDF.selectExpr(
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 "`F.Undergrad` as F_Undergrad",
 "`P.Undergrad` as P_Undergrad",
 'Outstate',
 "`Room.Board` as Room_Board",
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 "`S.F.Ratio` as S_F_Ratio",
 "`perc.alumni` as perc_alumi",
 'Expend',
"`Grad.Rate` as Grad_Rate" )
df.show()
# df.withColumn('Id_New',when(df.Rank <= 5,df.Id).otherwise('other')).drop(df.Id).select(col('Id_New').alias('Id'),col('Rank')).show()

+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+----------+------+---------+
|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumi|Expend|Grad_Rate|
+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+----------+------+---------+
|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|        12|  7041|       60|
|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30|     12.2|        16| 10527|       56|
|    Yes|1428|  1097|   336|       22|       50|       1036|         99|   11250|      3750|  400|    1165| 53|      66|     12.9|        30|  8735|       54|
|    Yes| 417|   349|   137|       60|       8

In [16]:
d = {}
# Fill in the entries one by one
df = df.drop('CN','Private')
for col in df.columns:
      d[col] = df.approxQuantile(col,[0.01,0.99],0.25)
      print(col+" done")

for col in df.columns:
    df = df.withColumn(col, \
    F.log(F.when(df[col] < d[col][0],d[col][0])\
    .when(df[col] > d[col][1], d[col][1])\
    .otherwise(df[col] ) +1).alias(col))
    print(col+" done")

df.show()
# assembler = VectorAssembler().setInputCols(df_new.columns).setOutputCol("features")
# transformed = assembler.transform(df_new)
# scaler = MinMaxScaler(inputCol="features",outputCol="scaledFeatures")
# scalerModel =  scaler.fit(transformed.select("features"))
# scaledData = scalerModel.transform(transformed)
# def extract(row):
#     return (row.pmid, )+tuple(row.scaledFeatures.toArray().tolist())
# final_data = scaledData.select("pmid","scaledFeatures").rdd.map(extract).toDF(df.columns)
# final_data.show()

Apps done
Accept done
Enroll done
Top10perc done
Top25perc done
F_Undergrad done
P_Undergrad done
Outstate done
Room_Board done
Books done
Personal done
PhD done
Terminal done
S_F_Ratio done
perc_alumi done
Expend done
Grad_Rate done
Apps done
Accept done
Enroll done
Top10perc done
Top25perc done
F_Undergrad done
P_Undergrad done
Outstate done
Room_Board done
Books done
Personal done
PhD done
Terminal done
S_F_Ratio done
perc_alumi done
Expend done
Grad_Rate done
+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|              Apps|            Accept|            Enroll|         Top10perc|         Top25perc|       F_Undergrad|       P_Undergrad|          Outstate|        Room_Board|             Books|          Person